In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings(action = 'ignore')
%matplotlib inline

# 데이터 분할
from sklearn.model_selection import train_test_split, cross_val_score, StratifiedKFold

# 모델
from xgboost import XGBClassifier 
from lightgbm import LGBMClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression

# 파라미터 최적화
from bayes_opt import BayesianOptimization

# 평가지표
from sklearn.metrics import log_loss

In [2]:
target = pd.read_csv('cust_train.csv', encoding = 'UTF-8')
label = target.LABEL

# pd w2v bayesian opt

In [18]:
train = pd.read_csv('pd_w2v_train.csv', encoding = 'UTF-8')
test = pd.read_csv('pd_w2v_test.csv', encoding = 'UTF-8')

X_train, X_val, y_train, y_val = train_test_split(train, label, test_size = 0.3, random_state = 516, stratify = label)

In [19]:
pbounds = {'n_estimators' : (10, 350),
           'learning_rate' : (0.03, 1.3),
           'max_depth' : (3, 10),
           'num_leaves' : (10, 200),
           'min_child_samples' : (20, 100),
           'min_child_weight' : (1, 10),
           'subsample' : (0.75, 0.95),
           'colsample_bytree' : (0.75, 0.95)}


def lgbm_opt(n_estimators, learning_rate, max_depth, num_leaves, min_child_samples, min_child_weight, subsample, colsample_bytree):
    
    params = {'n_estimators' : int(round(n_estimators)),
              'learning_rate': learning_rate,
              'max_depth' : int(round(max_depth)),
              'num_leaves' : int(round(num_leaves)),
              'min_child_samples' : int(round(min_child_samples)),
              'min_child_weight' : min_child_weight,
              'subsample': subsample,
              'colsample_bytree' : colsample_bytree,
              'n_jobs' : -1}
    
    lgbm = LGBMClassifier(**params)
    
    skfold = StratifiedKFold(n_splits = 4, shuffle = True, random_state = 516)
    
    score = cross_val_score(lgbm, X_train, y_train, cv = skfold, scoring = 'neg_log_loss', n_jobs=-1)
    
    return np.mean(score)

BO_lgbm = BayesianOptimization(f = lgbm_opt, pbounds = pbounds, random_state = 516)

BO_lgbm.maximize(init_points=40, n_iter=40)

max_params = BO_lgbm.max['params']
max_params['n_estimators'] = int(round(max_params['n_estimators']))
max_params['max_depth'] = int(round(max_params['max_depth']))
max_params['num_leaves'] = int(round(max_params['num_leaves']))
max_params['min_child_samples'] = int(round(max_params['min_child_samples']))
display(max_params)

# Step9. 최대화 하이퍼파라미터로 재학습
lgbm_tun = LGBMClassifier(**max_params)
lgbm_tun.fit(X_train, y_train)

skfold = StratifiedKFold(n_splits = 4, shuffle = True, random_state = 516)
scores = cross_val_score(lgbm_tun, X_val, y_val, cv = skfold, scoring = 'neg_log_loss', n_jobs=-1)

print(scores)
print(f'최대성능: {max(scores)}\n평균성능: {np.mean(scores)}')

|   iter    |  target   | colsam... | learni... | max_depth | min_ch... | min_ch... | n_esti... | num_le... | subsample |
-------------------------------------------------------------------------------------------------------------------------
|  1        | -1.696    |  0.9123   |  0.9297   |  3.781    |  65.34    |  8.135    |  294.8    |  113.0    |  0.789    |
|  2        | -5.896    |  0.7878   |  1.103    |  5.745    |  80.65    |  2.584    |  212.1    |  62.69    |  0.8719   |
|  3        | -2.458    |  0.8565   |  0.9781   |  6.776    |  20.55    |  5.861    |  240.8    |  67.81    |  0.8451   |
|  4        | -1.955    |  0.8397   |  0.5528   |  9.905    |  29.23    |  9.431    |  198.7    |  136.0    |  0.7549   |
|  5        | -1.583    |  0.879    |  0.9297   |  4.4      |  82.68    |  4.066    |  174.1    |  148.3    |  0.8592   |
|  6        | -1.31     |  0.9255   |  0.1295   |  5.885    |  57.35    |  1.101    |  91.62    |  161.4    |  0.8305   |
|  7        | -1.636    

|  41       | -1.298    |  0.75     |  0.03     |  6.156    |  59.37    |  10.0     |  313.4    |  94.79    |  0.8206   |
|  42       | -1.354    |  0.8655   |  0.3229   |  6.947    |  49.47    |  2.985    |  35.99    |  125.6    |  0.7527   |
|  43       | -1.443    |  0.8166   |  0.2888   |  5.91     |  24.86    |  2.176    |  230.8    |  151.2    |  0.8495   |
|  44       | -1.494    |  0.8348   |  0.6267   |  5.001    |  57.77    |  1.0      |  106.5    |  105.6    |  0.8654   |
|  45       | -1.471    |  0.8177   |  0.5701   |  4.173    |  64.35    |  9.235    |  323.9    |  131.7    |  0.8766   |
|  46       | -1.707    |  0.834    |  0.48     |  7.153    |  81.13    |  4.285    |  203.5    |  129.9    |  0.8603   |
|  47       | -1.578    |  0.8086   |  0.8623   |  5.239    |  44.15    |  4.885    |  94.93    |  130.9    |  0.8325   |
|  48       | -1.516    |  0.7766   |  0.547    |  7.292    |  55.24    |  2.845    |  64.27    |  146.7    |  0.9047   |
|  49       | -1.421    

{'colsample_bytree': 0.75,
 'learning_rate': 0.03,
 'max_depth': 6,
 'min_child_samples': 59,
 'min_child_weight': 10.0,
 'n_estimators': 313,
 'num_leaves': 95,
 'subsample': 0.8206383495380686}

[-1.3059556  -1.30588738 -1.30921729 -1.30877038]
최대성능: -1.3058873815455645
평균성능: -1.3074576653509726


In [ ]:
pd_w2v_pred = pd.DataFrame(lgbm_tun.predict_proba(test))
clnt_te = pd.DataFrame({'CLNT_ID':[i for i in range(263104,375864)]})
pd_w2v_pred = pd.concat([clnt_te, pd_w2v_pred],axis=1)
pd_w2v_pred.columns = ['CLNT_ID','F20','F30','F40','M20','M30','M40']
pd_w2v_pred.to_csv('pd_w2v_pred.csv',index=False)

# kwd w2v bayesian opt

In [8]:
train = pd.read_csv('kwd_w2v_train.csv', encoding = 'UTF-8')
test = pd.read_csv('kwd_w2v_test.csv', encoding = 'UTF-8')
train = train.iloc[:,1:]
test = test.iloc[:,1:]

X_train, X_val, y_train, y_val = train_test_split(train, label, test_size = 0.3, random_state = 516, stratify = label)

In [9]:
pbounds = {'n_estimators' : (50, 500),
           'learning_rate' : (0.01, 1.0),
           'max_depth' : (2, 10),
           'num_leaves' : (10, 200),
           'min_child_samples' : (20, 100),
           'min_child_weight' : (1, 15),
           'subsample' : (0.75, 0.95),
           'colsample_bytree' : (0.75, 0.95)}


def lgbm_opt(n_estimators, learning_rate, max_depth, num_leaves, min_child_samples, min_child_weight, subsample, colsample_bytree):
    
    params = {'n_estimators' : int(round(n_estimators)),
              'learning_rate': learning_rate,
              'max_depth' : int(round(max_depth)),
              'num_leaves' : int(round(num_leaves)),
              'min_child_samples' : int(round(min_child_samples)),
              'min_child_weight' : min_child_weight,
              'subsample': subsample,
              'colsample_bytree' : colsample_bytree,
              'n_jobs' : -1}
    
    lgbm = LGBMClassifier(**params)
    
    skfold = StratifiedKFold(n_splits = 4, shuffle = True, random_state = 516)
    
    score = cross_val_score(lgbm, X_train, y_train, cv = skfold, scoring = 'neg_log_loss', n_jobs=-1)
    
    return np.mean(score)

BO_lgbm = BayesianOptimization(f = lgbm_opt, pbounds = pbounds, random_state = 516)

BO_lgbm.maximize(init_points=40, n_iter=40)

max_params = BO_lgbm.max['params']
max_params['n_estimators'] = int(round(max_params['n_estimators']))
max_params['max_depth'] = int(round(max_params['max_depth']))
max_params['num_leaves'] = int(round(max_params['num_leaves']))
max_params['min_child_samples'] = int(round(max_params['min_child_samples']))
display(max_params)

# Step9. 최대화 하이퍼파라미터로 재학습
lgbm_tun = LGBMClassifier(**max_params)
lgbm_tun.fit(X_train, y_train)

skfold = StratifiedKFold(n_splits = 4, shuffle = True, random_state = 516)
scores = cross_val_score(lgbm_tun, X_val, y_val, cv = skfold, scoring = 'neg_log_loss', n_jobs=-1)

print(scores)
print(f'최대성능: {max(scores)}\n평균성능: {np.mean(scores)}')

|   iter    |  target   | colsam... | learni... | max_depth | min_ch... | min_ch... | n_esti... | num_le... | subsample |
-------------------------------------------------------------------------------------------------------------------------
|  1        | -1.364    |  0.9123   |  0.7113   |  2.892    |  65.34    |  12.1     |  426.9    |  113.0    |  0.789    |
|  2        | -1.734    |  0.7878   |  0.8463   |  5.137    |  80.65    |  3.463    |  317.5    |  62.69    |  0.8719   |
|  3        | -1.87     |  0.8565   |  0.7491   |  6.316    |  20.55    |  8.562    |  355.5    |  67.81    |  0.8451   |
|  4        | -1.739    |  0.8397   |  0.4175   |  9.892    |  29.23    |  14.12    |  299.8    |  136.0    |  0.7549   |
|  5        | -1.428    |  0.879    |  0.7114   |  3.6      |  82.68    |  5.769    |  267.1    |  148.3    |  0.8592   |
|  6        | -1.231    |  0.9255   |  0.08759  |  5.297    |  57.35    |  1.156    |  158.0    |  161.4    |  0.8305   |
|  7        | -1.5      

|  41       | -1.524    |  0.7672   |  0.7231   |  6.951    |  57.32    |  12.06    |  413.1    |  12.69    |  0.7878   |
|  42       | -1.275    |  0.7606   |  0.4312   |  4.009    |  60.52    |  9.797    |  191.5    |  10.68    |  0.934    |
|  43       | -1.267    |  0.95     |  0.01     |  7.308    |  78.67    |  1.0      |  163.3    |  177.5    |  0.75     |
|  44       | -2.026    |  0.8561   |  0.8537   |  7.369    |  56.04    |  1.051    |  159.2    |  184.4    |  0.9148   |
|  45       | -1.263    |  0.95     |  0.01     |  7.529    |  76.45    |  1.0      |  163.4    |  156.0    |  0.75     |
|  46       | -1.238    |  0.8077   |  0.1632   |  3.38     |  85.51    |  6.438    |  81.55    |  90.24    |  0.8362   |
|  47       | -1.511    |  0.7901   |  0.7173   |  4.17     |  56.62    |  2.877    |  413.8    |  56.4     |  0.7803   |
|  48       | -1.673    |  0.9405   |  0.4343   |  9.979    |  89.41    |  10.39    |  154.0    |  170.7    |  0.9048   |
|  49       | -1.253    

{'colsample_bytree': 0.8432589523901574,
 'learning_rate': 0.051055588083552005,
 'max_depth': 4,
 'min_child_samples': 51,
 'min_child_weight': 12.505756351754851,
 'n_estimators': 428,
 'num_leaves': 11,
 'subsample': 0.9335077173704511}

[-1.23775659 -1.23450612 -1.23108711 -1.23705596]
최대성능: -1.2310871116283506
평균성능: -1.2351014472670574


In [10]:
kwd_w2v_pred = pd.DataFrame(lgbm_tun.predict_proba(test))
clnt_te = pd.DataFrame({'CLNT_ID':[i for i in range(263104,375864)]})
kwd_w2v_pred = pd.concat([clnt_te, kwd_w2v_pred],axis=1)
kwd_w2v_pred.columns = ['CLNT_ID','F20','F30','F40','M20','M30','M40']
kwd_w2v_pred.to_csv('kwd2_w2v_pred.csv',index=False)

# brand w2v bayesian opt

In [ ]:
train = pd.read_csv('brand_w2v_train.csv', encoding = 'UTF-8')
test = pd.read_csv('brand_w2v_test.csv', encoding = 'UTF-8')

X_train, X_val, y_train, y_val = train_test_split(train, label, test_size = 0.3, random_state = 516, stratify = label)

In [ ]:
pbounds = {'n_estimators' : (10, 350),
           'learning_rate' : (0.03, 1.3),
           'max_depth' : (3, 10),
           'num_leaves' : (10, 200),
           'min_child_samples' : (20, 100),
           'min_child_weight' : (1, 10),
           'subsample' : (0.75, 0.95),
           'colsample_bytree' : (0.75, 0.95)}


def lgbm_opt(n_estimators, learning_rate, max_depth, num_leaves, min_child_samples, min_child_weight, subsample, colsample_bytree):
    
    
    params = {'n_estimators' : int(round(n_estimators)),
              'learning_rate': learning_rate,
              'max_depth' : int(round(max_depth)),
              'num_leaves' : int(round(num_leaves)),
              'min_child_samples' : int(round(min_child_samples)),
              'min_child_weight' : min_child_weight,
              'subsample': subsample,
              'colsample_bytree' : colsample_bytree,
              'n_jobs' : -1}
    
    lgbm = LGBMClassifier(**params)
    
    skfold = StratifiedKFold(n_splits = 4, shuffle = True, random_state = 516)
    
    score = cross_val_score(lgbm, X_train, y_train, cv = skfold, scoring = 'neg_log_loss', n_jobs=-1)
    
    return np.mean(score)

BO_lgbm = BayesianOptimization(f = lgbm_opt, pbounds = pbounds, random_state = 516)

BO_lgbm.maximize(init_points=40, n_iter=40)

max_params = BO_lgbm.max['params']
max_params['n_estimators'] = int(round(max_params['n_estimators']))
max_params['max_depth'] = int(round(max_params['max_depth']))
max_params['num_leaves'] = int(round(max_params['num_leaves']))
max_params['min_child_samples'] = int(round(max_params['min_child_samples']))
display(max_params)

# Step9. 최대화 하이퍼파라미터로 재학습
lgbm_tun = LGBMClassifier(**max_params)
lgbm_tun.fit(X_train, y_train)

skfold = StratifiedKFold(n_splits = 4, shuffle = True, random_state = 516)
scores = cross_val_score(lgbm_tun, X_val, y_val, cv = skfold, scoring = 'neg_log_loss', n_jobs=-1)

print(scores)
print(f'최대성능: {max(scores)}\n평균성능: {np.mean(scores)}')

In [ ]:
w2v_pred = pd.DataFrame(lgbm_clf.predict_proba(test))
clnt_te = pd.DataFrame({'CLNT_ID':[i for i in range(263104,375864)]})
w2v_pred = pd.concat([clnt_te, w2v_pred],axis=1)
w2v_pred.columns = ['CLNT_ID','F20','F30','F40','M20','M30','M40']
w2v_pred.to_csv('w2v_pred.csv',index=False)

# clac1 w2v bayesian opt

In [ ]:
train = pd.read_csv('clac1_w2v_train.csv', encoding = 'UTF-8')
test = pd.read_csv('clac1_w2v_test.csv', encoding = 'UTF-8')

X_train, X_val, y_train, y_val = train_test_split(train, label, test_size = 0.3, random_state = 516, stratify = label)

In [ ]:
pbounds = {'n_estimators' : (10, 350),
           'learning_rate' : (0.03, 1.3),
           'max_depth' : (3, 10),
           'num_leaves' : (10, 200),
           'min_child_samples' : (20, 100),
           'min_child_weight' : (1, 10),
           'subsample' : (0.75, 0.95),
           'colsample_bytree' : (0.75, 0.95)}


def lgbm_opt(n_estimators, learning_rate, max_depth, num_leaves, min_child_samples, min_child_weight, subsample, colsample_bytree):
    
    
    params = {'n_estimators' : int(round(n_estimators)),
              'learning_rate': learning_rate,
              'max_depth' : int(round(max_depth)),
              'num_leaves' : int(round(num_leaves)),
              'min_child_samples' : int(round(min_child_samples)),
              'min_child_weight' : min_child_weight,
              'subsample': subsample,
              'colsample_bytree' : colsample_bytree,
              'n_jobs' : -1}
    
    lgbm = LGBMClassifier(**params)
    
    skfold = StratifiedKFold(n_splits = 4, shuffle = True, random_state = 516)
    
    score = cross_val_score(lgbm, X_train, y_train, cv = skfold, scoring = 'neg_log_loss', n_jobs=-1)
    
    return np.mean(score)

BO_lgbm = BayesianOptimization(f = lgbm_opt, pbounds = pbounds, random_state = 516)

BO_lgbm.maximize(init_points=40, n_iter=40)

max_params = BO_lgbm.max['params']
max_params['n_estimators'] = int(round(max_params['n_estimators']))
max_params['max_depth'] = int(round(max_params['max_depth']))
max_params['num_leaves'] = int(round(max_params['num_leaves']))
max_params['min_child_samples'] = int(round(max_params['min_child_samples']))
display(max_params)

# Step9. 최대화 하이퍼파라미터로 재학습
lgbm_tun = LGBMClassifier(**max_params)
lgbm_tun.fit(X_train, y_train)

skfold = StratifiedKFold(n_splits = 4, shuffle = True, random_state = 516)
scores = cross_val_score(lgbm_tun, X_val, y_val, cv = skfold, scoring = 'neg_log_loss', n_jobs=-1)

print(scores)
print(f'최대성능: {max(scores)}\n평균성능: {np.mean(scores)}')

In [ ]:
w2v_pred = pd.DataFrame(lgbm_clf.predict_proba(test))
clnt_te = pd.DataFrame({'CLNT_ID':[i for i in range(263104,375864)]})
w2v_pred = pd.concat([clnt_te, w2v_pred],axis=1)
w2v_pred.columns = ['CLNT_ID','F20','F30','F40','M20','M30','M40']
w2v_pred.to_csv('w2v_pred.csv',index=False)

# clac2 w2v bayesian opt

In [ ]:
train = pd.read_csv('clac2_w2v_train.csv', encoding = 'UTF-8')
test = pd.read_csv('clac2_w2v_test.csv', encoding = 'UTF-8')

X_train, X_val, y_train, y_val = train_test_split(train, label, test_size = 0.3, random_state = 516, stratify = label)

In [ ]:
pbounds = {'n_estimators' : (10, 350),
           'learning_rate' : (0.03, 1.3),
           'max_depth' : (3, 10),
           'num_leaves' : (10, 200),
           'min_child_samples' : (20, 100),
           'min_child_weight' : (1, 10),
           'subsample' : (0.75, 0.95),
           'colsample_bytree' : (0.75, 0.95)}


def lgbm_opt(n_estimators, learning_rate, max_depth, num_leaves, min_child_samples, min_child_weight, subsample, colsample_bytree):
    
    
    params = {'n_estimators' : int(round(n_estimators)),
              'learning_rate': learning_rate,
              'max_depth' : int(round(max_depth)),
              'num_leaves' : int(round(num_leaves)),
              'min_child_samples' : int(round(min_child_samples)),
              'min_child_weight' : min_child_weight,
              'subsample': subsample,
              'colsample_bytree' : colsample_bytree,
              'n_jobs' : -1}
    
    lgbm = LGBMClassifier(**params)
    
    skfold = StratifiedKFold(n_splits = 4, shuffle = True, random_state = 516)
    
    score = cross_val_score(lgbm, X_train, y_train, cv = skfold, scoring = 'neg_log_loss', n_jobs=-1)
    
    return np.mean(score)

BO_lgbm = BayesianOptimization(f = lgbm_opt, pbounds = pbounds, random_state = 516)

BO_lgbm.maximize(init_points=40, n_iter=40)

max_params = BO_lgbm.max['params']
max_params['n_estimators'] = int(round(max_params['n_estimators']))
max_params['max_depth'] = int(round(max_params['max_depth']))
max_params['num_leaves'] = int(round(max_params['num_leaves']))
max_params['min_child_samples'] = int(round(max_params['min_child_samples']))
display(max_params)

# Step9. 최대화 하이퍼파라미터로 재학습
lgbm_tun = LGBMClassifier(**max_params)
lgbm_tun.fit(X_train, y_train)

skfold = StratifiedKFold(n_splits = 4, shuffle = True, random_state = 516)
scores = cross_val_score(lgbm_tun, X_val, y_val, cv = skfold, scoring = 'neg_log_loss', n_jobs=-1)

print(scores)
print(f'최대성능: {max(scores)}\n평균성능: {np.mean(scores)}')

In [ ]:
w2v_pred = pd.DataFrame(lgbm_clf.predict_proba(test))
clnt_te = pd.DataFrame({'CLNT_ID':[i for i in range(263104,375864)]})
w2v_pred = pd.concat([clnt_te, w2v_pred],axis=1)
w2v_pred.columns = ['CLNT_ID','F20','F30','F40','M20','M30','M40']
w2v_pred.to_csv('w2v_pred.csv',index=False)

# clac3 w2v bayesian opt

In [3]:
train = pd.read_csv('clac3_w2v_train.csv', encoding = 'UTF-8')
test = pd.read_csv('clac3_w2v_test.csv', encoding = 'UTF-8')

X_train, X_val, y_train, y_val = train_test_split(train, label, test_size = 0.3, random_state = 516, stratify = label)

In [ ]:
pbounds = {'n_estimators' : (100, 400),
           'learning_rate' : (0.01, 1.0),
           'max_depth' : (3, 13),
           'num_leaves' : (10, 200),
           'min_child_samples' : (10, 100),
           'min_child_weight' : (1, 20),
           'subsample' : (0.75, 0.95),
           'colsample_bytree' : (0.75, 0.95)}


def lgbm_opt(n_estimators, learning_rate, max_depth, num_leaves, min_child_samples, min_child_weight, subsample, colsample_bytree):
    
    
    params = {'n_estimators' : int(round(n_estimators)),
              'learning_rate': learning_rate,
              'max_depth' : int(round(max_depth)),
              'num_leaves' : int(round(num_leaves)),
              'min_child_samples' : int(round(min_child_samples)),
              'min_child_weight' : min_child_weight,
              'subsample': subsample,
              'colsample_bytree' : colsample_bytree,
              'n_jobs' : -1}
    
    lgbm = LGBMClassifier(**params)
    
    skfold = StratifiedKFold(n_splits = 4, shuffle = True, random_state = 516)
    
    score = cross_val_score(lgbm, X_train, y_train, cv = skfold, scoring = 'neg_log_loss', n_jobs=-1)
    
    return np.mean(score)

BO_lgbm = BayesianOptimization(f = lgbm_opt, pbounds = pbounds, random_state = 516)

BO_lgbm.maximize(init_points=40, n_iter=40)

max_params = BO_lgbm.max['params']
max_params['n_estimators'] = int(round(max_params['n_estimators']))
max_params['max_depth'] = int(round(max_params['max_depth']))
max_params['num_leaves'] = int(round(max_params['num_leaves']))
max_params['min_child_samples'] = int(round(max_params['min_child_samples']))
display(max_params)

# Step9. 최대화 하이퍼파라미터로 재학습
lgbm_tun = LGBMClassifier(**max_params)
lgbm_tun.fit(X_train, y_train)

skfold = StratifiedKFold(n_splits = 4, shuffle = True, random_state = 516)
scores = cross_val_score(lgbm_tun, X_val, y_val, cv = skfold, scoring = 'neg_log_loss', n_jobs=-1)

print(scores)
print(f'최대성능: {max(scores)}\n평균성능: {np.mean(scores)}')

|   iter    |  target   | colsam... | learni... | max_depth | min_ch... | min_ch... | n_esti... | num_le... | subsample |
-------------------------------------------------------------------------------------------------------------------------
|  1        | -1.388    |  0.9123   |  0.7113   |  4.115    |  61.01    |  16.06    |  351.3    |  113.0    |  0.789    |
|  2        | -16.8     |  0.7878   |  0.8463   |  6.922    |  78.23    |  4.343    |  278.3    |  62.69    |  0.8719   |
|  3        | -1.834    |  0.8565   |  0.7491   |  8.395    |  10.62    |  11.26    |  303.7    |  67.81    |  0.8451   |
|  4        | -1.663    |  0.8397   |  0.4175   |  12.86    |  20.38    |  18.8     |  266.5    |  136.0    |  0.7549   |
|  5        | -1.458    |  0.879    |  0.7114   |  5.0      |  80.52    |  7.472    |  244.8    |  148.3    |  0.8592   |
|  6        | -1.29     |  0.9255   |  0.08759  |  7.121    |  52.02    |  1.212    |  172.0    |  161.4    |  0.8305   |
|  7        | -1.47     

|  41       | -1.272    |  0.75     |  0.01     |  3.235    |  60.71    |  20.0     |  371.8    |  84.63    |  0.8548   |
|  42       | -1.381    |  0.8655   |  0.2383   |  8.639    |  43.16    |  5.19     |  122.9    |  125.6    |  0.7527   |
|  43       | -1.536    |  0.8239   |  0.4002   |  10.01    |  60.86    |  9.001    |  169.2    |  116.0    |  0.8351   |
|  44       | -16.75    |  0.95     |  1.0      |  3.0      |  50.86    |  20.0     |  304.3    |  10.0     |  0.75     |
|  45       | -1.261    |  0.7524   |  0.01     |  6.052    |  72.58    |  16.3     |  358.1    |  36.0     |  0.95     |
|  46       | -1.392    |  0.95     |  0.497    |  6.126    |  48.22    |  12.79    |  142.8    |  91.79    |  0.7872   |
|  47       | -1.261    |  0.8172   |  0.1225   |  4.064    |  28.82    |  9.897    |  160.5    |  135.8    |  0.7847   |
|  48       | -7.341    |  0.75     |  1.0      |  13.0     |  58.94    |  20.0     |  145.1    |  144.9    |  0.95     |
|  49       | -1.287    

In [6]:
display(max_params)
print(scores)
print(f'최대성능: {max(scores)}\n평균성능: {np.mean(scores)}')

{'colsample_bytree': 0.8432589523901574,
 'learning_rate': 0.051055588083552005,
 'max_depth': 5,
 'min_child_samples': 45,
 'min_child_weight': 16.614955048810156,
 'n_estimators': 352,
 'num_leaves': 11,
 'subsample': 0.9335077173704511}

[-1.27318992 -1.27103356 -1.27066252 -1.27162127]
최대성능: -1.2706625238476337
평균성능: -1.2716268163090936


In [7]:
clac3_w2v_pred = pd.DataFrame(lgbm_tun.predict_proba(test))
clnt_te = pd.DataFrame({'CLNT_ID':[i for i in range(263104,375864)]})
clac3_w2v_pred = pd.concat([clnt_te, clac3_w2v_pred],axis=1)
clac3_w2v_pred.columns = ['CLNT_ID','F20','F30','F40','M20','M30','M40']
clac3_w2v_pred.to_csv('clac3_w2v_pred.csv',index=False)